### 20. JSONデータの読み込み

In [ ]:
import json
import re

import requests

In [ ]:
with open('jawiki-country.json', 'r') as f:
    for line in f:
        data = json.loads(line)
        if data['title'] == 'イギリス':
            data_uk = data

text_uk = data_uk['text']
print(text_uk)

### 21. カテゴリ名を含む行を抽出

In [ ]:
category_lines = [line for line in text_uk.split('\n') if '[Category' in line]

In [ ]:
category_lines

### 22. カテゴリ名の抽出

In [ ]:
categories = []
for line in category_lines:
    category = line.strip('[').strip('Category:').strip(']').strip('*').strip('|')
    categories.append(category)

In [ ]:
categories

### 23. セクション構造

In [ ]:
for section in re.findall(r'(=+)([^=]+)(=+)\n', text_uk):
    print(f'{section[1]}\t{len(section[0]) - 1}')

### 24. ファイル参照の抽出

In [ ]:
results = re.findall(r'\[\[ファイル:(.+?)\|', text_uk)

In [ ]:
results

### 25. テンプレートの抽出

In [ ]:
info = re.findall(r'\{\{基礎情報.+?^\}\}', text_uk, re.MULTILINE + re.DOTALL)[0] # re.search().group()でも良い

result = {}
pattern = r'\|(.+?)\s*=\s*(.+)'
for e in re.findall(pattern, info):
    result[e[0]] = e[1]

In [ ]:
result

### 26. 強調マークアップの除去

In [ ]:
def remove_stress(s):
    return re.sub(r"\'+", '', s)

removed = {k: remove_stress(v) for k, v in result.items()}

In [ ]:
removed

### 27. 内部リンクの除去

In [ ]:
def remove_link(s):
    return re.sub(r'\[\[(.+?)\]\]', r'\1', s)

In [ ]:
remove_link('{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）')

In [ ]:
link_removed = {k: remove_link(remove_stress(v)) for k, v in result.items()}

In [ ]:
link_removed

### 28. MediaWikiマークアップの除去

In [ ]:
def remove_markups(text):
    # html タグ
    text = re.sub(r'<.+?>', '', text)
    # 外部リンク
    
    # テンプレート
    ''''For example,
    '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
    '標語': '{{lang|fr|Dieu et mon droit}}<br />（フランス語:Dieu et mon droit|神と我が権利）',
    '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}',
    '''
    text = re.sub(r'\{\{(?:lang|center|仮リンク)\|(?:(?:.+?)\|)*(.+?)\}\}', r'\1', text)
    # 日付
    '''
    '確立年月日2': '1707年{{0}}5月{{0}}1日',
    '''
    text = re.sub(r'\{\{\d\}\}', '', text)
    # ファイル
    '''For example,
    '国章画像': 'ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章',
    '''
    text = re.sub(r'ファイル:(.+?)\|(.+)', r'\1', text)
    return text

In [ ]:
result_29 = {}
for k, v in result.items():
    _removed = remove_stress(v)
    _removed = remove_link(_removed)
    result_29[k] = remove_markups(_removed)

In [ ]:
result_29

### 29. 国旗画像のURLを取得する

In [ ]:
def get_url(data):
    filename = data['国旗画像'].replace(' ', '_')
    url = "https://commons.wikimedia.org/w/api.php?action=query&format=json&prop=imageinfo&iiprop=url&titles=File:{}".format(filename)
    res = requests.get(url)
    image_url = re.search(r'"url":"(https?://.+?)"', res.text).group(1)
    return image_url

In [ ]:
get_url(result_29)